1. Importing Libraries
2. Preprocessing
3. KRW-BTC candle chart
      - plotly
4. KRW-BTC,SOL,XRP,DOGE daily price returns
      - graph
      - histogram
5. KRW-BTC,SOL,XRP,DOGE Correlation
      - scatter matrix
      - heatmap
6. KRW-BTC,SOL,XRP,DOGE risk and returns
      - scatter 
7. Predicting closing price KRW-BTC
      - LSTM

In [1]:
%load_ext autoreload
%autoreload 2

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import ta.momentum
import ta.volatility
import ta.volume
import ta.trend
import requests
from ta.utils import dropna

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from modules import *
import time

# Preprocessing

In [2]:
api=UpbitAPI()

In [14]:
df = api.day_candle(count=1500)

def applytech(df:pd.DataFrame):
    df.sort_index(inplace=True)
    df['bol_upper']=ta.volatility.bollinger_hband(df['trade_price'])
    df['bol_lower']=ta.volatility.bollinger_lband(df['trade_price'])
    df['rsi']=ta.momentum.rsi(df['trade_price'])
    df['stochastic_oscillators']=ta.momentum.StochasticOscillator(df['high_price'],df['low_price'],df['trade_price']).stoch_signal()
    df['12ema']=ta.trend.ema_indicator(df['trade_price'],window=12)
    df['26ema']=ta.trend.ema_indicator(df['trade_price'],window=26)
    df['macd']=ta.trend.macd(df['trade_price'])
    df['macd_signal']=ta.trend.macd_signal(df['trade_price'])
    df['macd_oscillators']=ta.trend.macd_diff(df['trade_price'])
    df['Daily return']=df['trade_price'].pct_change()
    df.rename(columns = {'opening_price':'open','trade_price':'close','high_price':'high','low_price':'low','candle_acc_trade_volume':'volume'},inplace=True)
    return df
df=applytech(df)
df

,open,high,low,close,candle_acc_trade_price,volume,bol_upper,bol_lower,rsi,stochastic_oscillators,12ema,26ema,macd,macd_signal,macd_oscillators,Daily return
datetime64_kst,,,,,,,,,,,,,,,,
2019-01-26 09:00:00,3957000.0,4039000.0,3934000.0,3963000.0,1.498534e+10,3770.714418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-27 09:00:00,3963000.0,3975000.0,3900000.0,3951000.0,1.137707e+10,2879.052233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003028
2019-01-28 09:00:00,3950000.0,3952000.0,3775000.0,3828000.0,2.230458e+10,5807.396995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.031131
2019-01-29 09:00:00,3828000.0,3841000.0,3730000.0,3795000.0,1.528309e+10,4037.590559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008621
2019-01-30 09:00:00,3796000.0,3855000.0,3769000.0,3810000.0,2.903848e+10,7619.508244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-08 09:00:00,29579000.0,29660000.0,29000000.0,29023000.0,1.616345e+11,5501.293981,3.254274e+07,2.915826e+07,38.984981,7.058683,3.011507e+07,3.007298e+07,42084.260039,390977.265464,-348893.005424,-0.018797
2023-03-09 09:00:00,29018000.0,29320000.0,27415000.0,27604000.0,2.807600e+11,9841.876226,3.281283e+07,2.848447e+07,29.840428,5.731170,2.972875e+07,2.989010e+07,-161346.261453,280512.560080,-441858.821533,-0.048892
2023-03-10 09:00:00,27604000.0,27605000.0,26707000.0,27343000.0,2.633127e+11,9730.632240,3.295842e+07,2.789428e+07,28.515510,6.027266,2.936171e+07,2.970142e+07,-339710.957648,156467.856535,-496178.814183,-0.009455


## KRW-BTC candle chart


    - plotly

In [5]:
# set plot
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
               vertical_spacing=0.02, 
               row_width=[0.1,0.2,0.1,0.6])

# row 1, col 1
fig.add_trace(go.Candlestick(x=df.index,
                    open=df['open'], high=df['high'],
                    low=df['low'], close=df['close'],showlegend=False),row=1, col=1,)
fig.add_trace(go.Scatter(x=df.index,y=df['bol_upper'],name='bol_upper',line=dict(color='royalblue', width=1, dash='dash')), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['bol_lower'],name='bol_lower',line=dict(color='firebrick', width=1, dash='dash')), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['26ema'],name='26ema',line=dict(color='blue', width=1, )), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['12ema'],name='12ema',line=dict(color='#ff3399', width=1, )), row=1,
                 col=1)

# row 2, col 1
green_volume_df = df[df['close'] > df['open']]
red_volume_df = df[df['close'] < df['open']]
fig.add_trace(go.Bar(x=red_volume_df.index, y=red_volume_df.volume, showlegend=False, marker_color='#ef5350'), row=2,
                 col=1)
fig.add_trace(go.Bar(x=green_volume_df.index, y=green_volume_df.volume, showlegend=False, marker_color='#26a69a'),
                 row=2, col=1)

# row 3, col 1
fig.add_trace(go.Scatter(x=df.index,y=df['rsi'],name='rsi',line=dict(color='blue', width=1, )), row=3,
                 col=1)
fig.add_trace(go.Bar(x=df.index,y=df['macd_oscillators'],name='macd_osc[12,26,9]'), row=3,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['macd'],name='macd',), row=3,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['macd_signal'],name='macd_signal',), row=3,
                 col=1)

# row 4, col 1
fig.add_trace(go.Scatter(x=df.index,y=df['stochastic_oscillators'],name='stochastic_oscillators',line=dict(color='blue', width=1, )), row=3,
                 col=1)


# label and plot
fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'KRW-BTC', yaxis_title=f'Price')
fig.update_yaxes(title_text=f'Volume', row=2, col=1)
fig.update_yaxes(title_text=f'MACD', row=3, col=1)
fig.update_yaxes(title_text=f'stochastic_oscillators', row=4, col=1)
fig.update_xaxes(title_text='Date', row=2)
fig.show()

## KRW-BTC,SOL,XRP,DOGE daily price return
    - graph

In [6]:
# get data
df_BTC=applytech(api.min_candle(unit=10,market="KRW-BTC",count=1000))

df_SOL=applytech(api.min_candle(unit=10,market="KRW-SOL",count=1000))

df_XRP=applytech(api.min_candle(unit=10,market="KRW-XRP",count=1000))

df_DOGE=applytech(api.min_candle(unit=10,market="KRW-DOGE",count=1000))


# x-axis by time series
fig=make_subplots(rows=2, cols=2, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=df_BTC.index,y=df_BTC['Daily return'],name='KRW_BTC',line=dict(color='royalblue', width=0.5)), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df_SOL.index,y=df_SOL['Daily return'],name='KRW_SOL',line=dict(color='royalblue', width=0.5)), row=1,
                 col=2)
fig.add_trace(go.Scatter(x=df_XRP.index,y=df_XRP['Daily return'],name='KRW_XRP',line=dict(color='royalblue', width=0.5)), row=2,
                 col=1)
fig.add_trace(go.Scatter(x=df_DOGE.index,y=df_DOGE['Daily return'],name='KRW_DOGE',line=dict(color='royalblue', width=0.5)), row=2,
                 col=2)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'Daily return', yaxis_title=f'pct_change')
fig.update_yaxes(title_text=f'KRW_BTC', row=1, col=1)
fig.update_yaxes(title_text=f'KRW_SOL', row=1, col=2)
fig.update_yaxes(title_text=f'KRW_XRP', row=2, col=1)
fig.update_yaxes(title_text=f'KRW_DOGE', row=2, col=2)

    - histogram

In [7]:
# x-axis by values(pct_change)

fig=make_subplots(rows=2, cols=2, vertical_spacing=0.02)

fig.add_trace(go.Histogram(x=df_BTC['Daily return'],name='KRW_BTC'), row=1,
                 col=1)
fig.add_trace(go.Histogram(x=df_SOL['Daily return'],name='KRW_SOL',), row=1,
                 col=2)
fig.add_trace(go.Histogram(x=df_XRP['Daily return'],name='KRW_XRP',), row=2,
                 col=1)
fig.add_trace(go.Histogram(x=df_DOGE['Daily return'],name='KRW_DOGE',), row=2,
                 col=2)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'Daily return', yaxis_title=f'pct_change')
fig.update_yaxes(title_text=f'KRW_BTC', row=1, col=1)
fig.update_yaxes(title_text=f'KRW_SOL', row=1, col=2)
fig.update_yaxes(title_text=f'KRW_XRP', row=2, col=1)
fig.update_yaxes(title_text=f'KRW_DOGE', row=2, col=2)

## KRW-BTC,SOL,XRP,DOGE returns Correlation
    - Scatter matrix

In [8]:
# Splom = scatter matrix in plotly
fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='KRW_BTC',
                                 values=df_BTC['Daily return']),
                            dict(label='KRW_SOL',
                                 values=df_SOL['Daily return']),
                            dict(label='KRW_XRP',
                                 values=df_XRP['Daily return']),
                            dict(label='KRW_DOGE',
                                 values=df_DOGE['Daily return'])],
                )
                )


fig.update_layout(
    title='Daily return',
    dragmode='select',
    width=600,
    height=600,
    hovermode='closest',
)

fig.show()

    - Correlation heatmap

In [9]:
# get correlation matrix
stocks_returns=pd.concat([df_BTC['Daily return'],df_SOL['Daily return'],df_XRP['Daily return'],df_DOGE['Daily return']],axis=1)
stocks_close=pd.concat([df_BTC['close'],df_SOL['close'],df_XRP['close'],df_DOGE['close']],axis=1)
stocks_returns.columns=['KRW-BTC','KRW-SOL','KRW-XRP','KRW-DOGE']
stocks_close.columns=['KRW-BTC','KRW-SOL','KRW-XRP','KRW-DOGE']

returns_corr=stocks_returns.corr()
close_corr=stocks_close.corr()


fig=make_subplots(rows=2, cols=1, vertical_spacing=0.2)
fig.add_trace(go.Heatmap(
                    z=returns_corr,
                    x=['BTC','SOL','XRP','DOGE'],
                    y=['BTC','SOL','XRP','DOGE'],
                    text=round(returns_corr,3),
                    texttemplate="%{text}",
                    textfont={"size":20}),row=1,col=1)
fig.add_trace(go.Heatmap(
                    z=close_corr,
                    x=['BTC','SOL','XRP','DOGE'],
                    y=['BTC','SOL','XRP','DOGE'],
                    text=round(close_corr,3),
                    texttemplate="%{text}",
                    textfont={"size":20}),row=2,col=1)

fig.update_yaxes(title_text=f'returns_corr', row=1, col=1)
fig.update_yaxes(title_text=f'close_corr', row=2, col=1)
fig.show()

## KRW-BTC,SOL,XRP,DOGE risk and returns
     - scatter 

In [10]:
fig=go.Figure()
for i in stocks_returns.columns:
    fig.add_trace(go.Scatter(x=[stocks_returns[i].mean()],
                                y=[stocks_returns[i].std()],
                                mode='markers',
                                name=i
                                )) # hover text goes here

fig.update_layout(title='Risk-Returns')
fig.update_yaxes(title_text=f'risk')
fig.update_xaxes(title_text=f'returns')
fig.show()

## Predicting closing price KRW-BTC
      - LSTM

In [44]:
df_BTC=api.min_candle(unit=240,market="KRW-DOGE",count=2000)
df_BTC.sort_index(inplace=True)
df_BTC=applytech(df_BTC)


data = df_BTC.filter(['Daily return','rsi','volume','stochastic_oscillators'])
data['Daily return']=data['Daily return'].shift(1,fill_value=0)
data.dropna(inplace=True)



df_y=data.apply(lambda x: 1 if x['Daily return']>=0 else 0,axis=1)

data.drop(columns='Daily return',inplace=True)
print(data,df_y)
dataset = data.values
dataset_y=df_y.values

training_data_len = int(np.ceil( len(dataset) * .9 ))
training_data_len


                           rsi        volume  stochastic_oscillators
datetime64_kst                                                      
2022-04-14 13:00:00  48.140742  1.376551e+08               57.142857
2022-04-14 17:00:00  60.142091  4.562251e+08               63.585434
2022-04-14 21:00:00  49.799444  2.346163e+08               55.490196
2022-04-15 01:00:00  51.592447  1.313200e+08               53.109244
2022-04-15 05:00:00  58.047180  9.380275e+07               51.428571
...                        ...           ...                     ...
2023-03-12 01:00:00  36.982351  2.011762e+07               20.909091
2023-03-12 05:00:00  41.117784  2.366917e+07               26.400453
2023-03-12 09:00:00  39.445197  3.867044e+07               37.977089
2023-03-12 13:00:00  38.877494  3.139422e+07               44.345117
2023-03-12 17:00:00  39.810389  8.164688e+06               48.309179

[1985 rows x 3 columns] datetime64_kst
2022-04-14 13:00:00    1
2022-04-14 17:00:00    0
2022-04-14 21

1787

In [45]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.46309725, 0.03171525, 0.58134877],
       [0.6140984 , 0.10659957, 0.64981685],
       [0.48396707, 0.05450732, 0.56378522],
       ...,
       [0.35368977, 0.00844753, 0.37766578],
       [0.34654693, 0.00673715, 0.4453416 ],
       [0.35828463, 0.00127673, 0.48746941]])

In [46]:
train_data = scaled_data[0:int(training_data_len), :]
train_data_y=dataset_y[0:int(training_data_len)]
print(train_data.shape)
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, :])
    y_train.append(train_data_y[i])


    

x_train, y_train = np.array(x_train), np.array(y_train)
y_train=np.reshape(y_train,(y_train.shape[0],1))
print(x_train.shape,y_train.shape)
#x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


(1787, 3)
(1727, 60, 3) (1727, 1)


In [56]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout, Activation
# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2]),activation='softsign',dropout=0.2))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False,activation='softsign',dropout=0.2))
model.add(Dense(16))
model.add(Dense(1))
model.add(Activation('tanh'))

# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
# Train the model
model.fit(x_train, y_train,batch_size=x_train.shape[0]//50,epochs=5)

Epoch 1/5
51/51 [==============================] - 8s 83ms/step - loss: 0.2704 - mse: 0.2704
Epoch 2/5
51/51 [==============================] - 4s 74ms/step - loss: 0.2504 - mse: 0.2504
Epoch 3/5
51/51 [==============================] - 4s 74ms/step - loss: 0.2514 - mse: 0.2514
Epoch 4/5
51/51 [==============================] - 4s 78ms/step - loss: 0.2506 - mse: 0.2506
Epoch 5/5
51/51 [==============================] - 4s 75ms/step - loss: 0.2490 - mse: 0.2490


In [57]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset_y[training_data_len:]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, :])
    
# Convert the data to a numpy array
x_test = np.array(x_test)
print(x_test.shape)

# Reshape the data
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2] ))
#print(x_test.shape)
# Get the models predicted price values 
predictions = model.predict(x_test)
print(predictions.shape)
#predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

(198, 60, 3)
7/7 [==============================] - 1s 31ms/step
(198, 1)


0.49364059647544634

In [58]:
print(training_data_len)

1787


In [59]:
predictions,y_test

(array([[0.5366139 ],
        [0.5354612 ],
        [0.5342341 ],
        [0.53289574],
        [0.5309734 ],
        [0.5286534 ],
        [0.5263535 ],
        [0.52456367],
        [0.523826  ],
        [0.52406734],
        [0.5241219 ],
        [0.52483284],
        [0.52554286],
        [0.52581394],
        [0.5256137 ],
        [0.52515316],
        [0.5246064 ],
        [0.524121  ],
        [0.5233215 ],
        [0.5222168 ],
        [0.52108294],
        [0.5195974 ],
        [0.5181508 ],
        [0.51685995],
        [0.5154124 ],
        [0.5140677 ],
        [0.5131408 ],
        [0.51238424],
        [0.511538  ],
        [0.51078665],
        [0.5108369 ],
        [0.5110115 ],
        [0.51113296],
        [0.51162016],
        [0.51292795],
        [0.5150106 ],
        [0.51685697],
        [0.51857424],
        [0.52044237],
        [0.5221728 ],
        [0.5232559 ],
        [0.5242437 ],
        [0.52507854],
        [0.5255513 ],
        [0.52583945],
        [0

In [60]:
df_BTC_pred=pd.DataFrame(index=df_BTC.index[-len(predictions):],columns=['up_down'],data=predictions)
df_BTC=pd.DataFrame(index=df_BTC.index[-len(y_test):],columns=['up_down'],data=y_test)
df_BTC_pred,df_BTC

(                      up_down
 datetime64_kst               
 2023-02-07 21:00:00  0.536614
 2023-02-08 01:00:00  0.535461
 2023-02-08 05:00:00  0.534234
 2023-02-08 09:00:00  0.532896
 2023-02-08 13:00:00  0.530973
 ...                       ...
 2023-03-12 01:00:00  0.506137
 2023-03-12 05:00:00  0.506541
 2023-03-12 09:00:00  0.507180
 2023-03-12 13:00:00  0.507478
 2023-03-12 17:00:00  0.507629
 
 [198 rows x 1 columns],
                      up_down
 datetime64_kst              
 2023-02-07 21:00:00        1
 2023-02-08 01:00:00        0
 2023-02-08 05:00:00        1
 2023-02-08 09:00:00        1
 2023-02-08 13:00:00        1
 ...                      ...
 2023-03-12 01:00:00        0
 2023-03-12 05:00:00        1
 2023-03-12 09:00:00        1
 2023-03-12 13:00:00        0
 2023-03-12 17:00:00        0
 
 [198 rows x 1 columns])

In [61]:
acc=pd.DataFrame(pd.concat([df_BTC['up_down'],df_BTC_pred['up_down']],axis=1).values,columns=['y','ypred'])
acc=acc.dropna(axis=0)
acc['correct']=acc.apply(lambda x: 1 if (((x['y']-0.5)*(x['ypred']-0.5))>0) else 0,axis=1)

score=(acc['correct'].sum()/len(acc))
score

0.6464646464646465

In [26]:
df_BTC_pred.tail(15),data.tail(15)

(                      up_down
 datetime64_kst               
 2023-03-10 09:00:00  0.561043
 2023-03-10 13:00:00  0.559337
 2023-03-10 17:00:00  0.557952
 2023-03-10 21:00:00  0.556763
 2023-03-11 01:00:00  0.555995
 2023-03-11 05:00:00  0.555919
 2023-03-11 09:00:00  0.556147
 2023-03-11 13:00:00  0.556387
 2023-03-11 17:00:00  0.556202
 2023-03-11 21:00:00  0.555902
 2023-03-12 01:00:00  0.556316
 2023-03-12 05:00:00  0.557260
 2023-03-12 09:00:00  0.558605
 2023-03-12 13:00:00  0.559252
 2023-03-12 17:00:00  0.558961,
                      Daily return        rsi        volume  \
 datetime64_kst                                               
 2023-03-10 09:00:00     -0.031419  20.170127  1.672727e+08   
 2023-03-10 13:00:00     -0.005593  19.867717  6.831978e+07   
 2023-03-10 17:00:00     -0.002250  17.594973  1.331490e+08   
 2023-03-10 21:00:00     -0.018038  22.376333  1.020152e+08   
 2023-03-11 01:00:00      0.009185  26.400407  2.897422e+07   
 2023-03-11 05:00:00      0.007

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Raw Data",
    x=df_BTC.index, y=df_BTC['close'],
    
))
fig.add_trace(go.Scatter(
    name="predicton Data",
     x=df_BTC_pred.index, y=df_BTC_pred['close']
))

fig.update_xaxes(
    rangeslider_visible=True)
fig.show()

KeyError: 'close'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df_BTC.index,
                    open=df_BTC['open'], high=df_BTC['high'],
                    low=df_BTC['low'], close=df_BTC['close'],showlegend=False))
fig.add_trace(go.Candlestick(x=df_BTC_pred.index,
                    open=df_BTC_pred['open'], high=df_BTC_pred['high'],
                    low=df_BTC_pred['low'], close=df_BTC_pred['close'],showlegend=False))
fig.update_xaxes(
    rangeslider_visible=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Raw Data",
    x=df_BTC.index, y=df_BTC['Daily return'],
    
))
fig.add_trace(go.Scatter(
    name="predicton Data",
     x=df_BTC_pred.index, y=df_BTC_pred['Daily return']
))

fig.update_xaxes(
    rangeslider_visible=True)
fig.show()

In [8]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout, Activation
def BoB_demo(coin,batchsize=50):

    '''
    df_BTC=api.min_candle(unit=240,market="KRW-DOGE",count=2000)
    df_BTC.sort_index(inplace=True)
    df_BTC=applytech(df_BTC)


    data = df_BTC.filter(['Daily return','rsi','volume','stochastic_oscillators'])
    data['Daily return']=data['Daily return'].shift(1,fill_value=0)
    data.dropna(inplace=True)

    dataset = data.values
    dataset_y=df_y.values

    training_data_len = int(np.ceil( len(dataset) * .9 ))
    training_data_len


    df_y=data.apply(lambda x: 1 if x['Daily return']>=0 else 0,axis=1)

    print(data,df_y)
    dataset = data.values
    dataset_y=df_y.values

    training_data_len = int(np.ceil( len(dataset) * .9 ))
    training_data_len

    train_data = scaled_data[0:int(training_data_len), :]
train_data_y=dataset_y[0:int(training_data_len)]
print(train_data.shape)
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, :])
    y_train.append(train_data_y[i])


    

x_train, y_train = np.array(x_train), np.array(y_train)
y_train=np.reshape(y_train,(y_train.shape[0],1))
print(x_train.shape,y_train.shape)

    '''
    df=api.min_candle(unit=240,market="KRW-DOGE",count=2000)
    df.sort_index(inplace=True)
    df=applytech(df)
    data = df.filter(['Daily return','rsi','volume','stochastic_oscillators'])
    data['Daily return']=data['Daily return'].shift(1,fill_value=0)
    data.dropna(inplace=True)
    
    dataset = data.values
    training_data_len = int(np.ceil( len(dataset) * .9 ))
    df_y=data.apply(lambda x: 1 if x['Daily return']>=0 else 0,axis=1)
    
    dataset = data.values
    dataset_y=df_y.values
    
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    train_data = scaled_data[0:int(training_data_len), :]
    train_data_y=dataset_y[0:int(training_data_len)]

    x_train = []
    y_train = []

    for i in range(batchsize, len(train_data)):
        x_train.append(train_data[i-batchsize:i, :])
        y_train.append(train_data_y[i])

        

    x_train, y_train = np.array(x_train), np.array(y_train)
    y_train=np.reshape(y_train,(y_train.shape[0],1))
    
    breakpoint()
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2]),activation='ReLU'))
    model.add(LSTM(64, return_sequences=False,activation='ReLU'))
    model.add(Dense(16))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))


    model.fit(x_train, y_train,batch_size=x_train.shape[0]//50,epochs=5)
    model.save('model.h5')
    test_data = scaled_data[training_data_len - batchsize: , :]

    '''
    test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset_y[training_data_len:]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, :])
    
# Convert the data to a numpy array
x_test = np.array(x_test)
print(x_test.shape)

# Reshape the data
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2] ))
#print(x_test.shape)
# Get the models predicted price values 
predictions = model.predict(x_test)
print(predictions.shape)
#predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse
    '''

    x_test = []
    y_test = dataset_y[training_data_len:, :]
    for i in range(batchsize, len(test_data)):
        x_test.append(test_data[i-batchsize:i, :])
        

    x_test = np.array(x_test)
    
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)


    # rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))

    '''
    df_BTC_pred=pd.DataFrame(index=df_BTC.index[-len(predictions):],columns=['up_down'],data=predictions)
    df_BTC=pd.DataFrame(index=df_BTC.index[-len(y_test):],columns=['up_down'],data=y_test)
    '''
    df_pred=pd.DataFrame(index=df_BTC.index[-len(predictions):],columns=['up_down'],data=predictions)
    df=pd.DataFrame(index=df_BTC.index[-len(y_test):],columns=['up_down'],data=y_test)

    '''
    acc=pd.DataFrame(pd.concat([df_BTC['up_down'],df_BTC_pred['up_down']],axis=1).values,columns=['y','ypred'])
    acc=acc.dropna(axis=0)
    acc['correct']=acc.apply(lambda x: 1 if (((x['y']-0.5)*(x['ypred']-0.5))>0) else 0,axis=1)

    score=(acc['correct'].sum()/len(acc))
    '''
    acc=pd.DataFrame(pd.concat([df['Daily return'],df_pred['Daily return']],axis=1).values,columns=['y','ypred'])
    acc=acc.dropna(axis=0)
    acc['correct']=acc.apply(lambda x: 1 if ((x.y*x.ypred)>0) else 0,axis=1)

    score=(acc['correct'].sum()/len(acc))

    return score


coins=api.top_markets_by_trade_price(keyword='KRW').iloc[2:13]['market'].values
board=dict()
for i in coins:
    board[i]=BoB_demo(i)
print(board)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [16]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout, Activation

def Bob_demo(coin="KRW-DOGE",batchsize=50):
    df=api.min_candle(unit=240,market="KRW-DOGE",count=2000)
    df.sort_index(inplace=True)
    df=applytech(df)


    data = df.filter(['Daily return','rsi','volume','stochastic_oscillators'])
    data['Daily return']=data['Daily return'].shift(1,fill_value=0)
    data.dropna(inplace=True)



    df_y=data.apply(lambda x: 1 if x['Daily return']>=0 else 0,axis=1)

    dataset = data.values
    dataset_y=df_y.values

    training_data_len = int(np.ceil( len(dataset) * .9 ))



    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    scaled_data

    train_data = scaled_data[0:int(training_data_len), :]
    train_data_y=dataset_y[0:int(training_data_len)]

    x_train = []
    y_train = []

    for i in range(batchsize, len(train_data)):
        x_train.append(train_data[i-batchsize:i, :])
        y_train.append(train_data_y[i])


        

    x_train, y_train = np.array(x_train), np.array(y_train)
    y_train=np.reshape(y_train,(y_train.shape[0],1))
    print(x_train.shape,y_train.shape)


    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2]),activation='ReLU'))
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=False,activation='ReLU'))
    model.add(Dense(16))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(x_train, y_train,batch_size=x_train.shape[0]//batchsize,epochs=5)

    test_data = scaled_data[training_data_len - batchsize: , :]
    # Create the data sets x_test and y_test
    x_test = []
    y_test = dataset_y[training_data_len:]
    for i in range(batchsize, len(test_data)):
        x_test.append(test_data[i-batchsize:i, :])
        
    # Convert the data to a numpy array
    x_test = np.array(x_test)


    # Reshape the data
    #x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2] ))
    #print(x_test.shape)
    # Get the models predicted price values 
    predictions = model.predict(x_test)

    #predictions = scaler.inverse_transform(predictions)

    # Get the root mean squared error (RMSE)
    rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))


    df_pred=pd.DataFrame(index=df.index[-len(predictions):],columns=['up_down'],data=predictions)
    df=pd.DataFrame(index=df.index[-len(y_test):],columns=['up_down'],data=y_test)


    acc=pd.DataFrame(pd.concat([df['up_down'],df_pred['up_down']],axis=1).values,columns=['y','ypred'])
    acc=acc.dropna(axis=0)
    acc['correct']=acc.apply(lambda x: 1 if (((x['y']-0.5)*(x['ypred']-0.5))>0) else 0,axis=1)

    score=(acc['correct'].sum()/len(acc))

    return score

Bob_demo("KRW-BTC",50)

(1737, 50, 4) (1737, 1)
Epoch 1/5
52/52 [==============================] - 6s 56ms/step - loss: 0.6913 - accuracy: 0.5406
Epoch 2/5
52/52 [==============================] - 3s 55ms/step - loss: 0.6908 - accuracy: 0.5406
Epoch 3/5
52/52 [==============================] - 3s 58ms/step - loss: 0.6908 - accuracy: 0.5406
Epoch 4/5
52/52 [==============================] - 3s 55ms/step - loss: 0.6906 - accuracy: 0.5406
Epoch 5/5
7/7 [==============================] - 0s 25ms/step


0.6464646464646465

In [10]:
print(score)

0.6464646464646465
